In [4]:
from __future__ import annotations

from typing import List, Optional
from functools import wraps
import logging
from pydantic import BaseModel, EmailStr, field_validator, Field

logging.basicConfig(
  level=logging.INFO,
  format="%(asctime)s [%(levelname)s] %(message)s",
)

class BookNotAvailable(Exception):
  """Книга недоступна для выдачи (уже на руках или нет в библиотеке)."""

def log_operation(action: str):
  def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
      logging.info("Operation %s: started. args=%s kwargs=%s", action, args, kwargs)
      result = func(*args, **kwargs)
      logging.info("Operation %s: finished. result=%s", action, result)
      return result
    return wrapper
  return decorator


class Book(BaseModel):
  title: str
  author: str
  year: int
  available: bool = True
  categories: List[str] = Field(default_factory=list)

  @field_validator("year")
  @classmethod
  def year_must_be_reasonable(cls, v: int) -> int:
    if v < 0 or v > 2100:
      raise ValueError("year должен быть в диапазоне [0, 2100]")
    return v

  @field_validator("categories")
  @classmethod
  def validate_categories(cls, cats: List[str]) -> List[str]:
    cleaned = [c.strip() for c in cats]
    if any(len(c) == 0 for c in cleaned):
      raise ValueError("categories не должны содержать пустых строк")
    seen = set()
    unique: List[str] = []
    for c in cleaned:
      if c.lower() not in seen:
        seen.add(c.lower())
        unique.append(c)
    return unique


class User(BaseModel):
  name: str
  email: EmailStr
  membership_id: str


class Library(BaseModel):
  books: List[Book] = []
  users: List[User] = []

  def total_books(self) -> int:
    return len(self.books)


def add_book(library: Library, book: Book) -> Book:
  library.books.append(book)
  return book


def find_book(library: Library, title: str, author: Optional[str] = None) -> Optional[Book]:

  title_low = title.strip().lower()
  author_low = author.strip().lower() if author else None

  for b in library.books:
    if b.title.strip().lower() == title_low and (author_low is None or b.author.strip().lower() == author_low):
      return b
  return None


@log_operation("borrow")
def is_book_borrow(library: Library, title: str) -> Book:
  book = find_book(library, title=title)
  if book is None:
    raise BookNotAvailable(f"Книга '{title}' не найдена в библиотеке.")
  if not book.available:
    raise BookNotAvailable(f"Книга '{title}' уже на руках.")

  book.available = False
  return book


@log_operation("return")
def return_book(library: Library, title: str) -> Book:
  book = find_book(library, title=title)
  if book is None:
    raise BookNotAvailable(f"Книга '{title}' не найдена в библиотеке.")
  book.available = True
  return book

if __name__ == "__main__":
  lib = Library()
  user = User(name="Alice", email="alice@example.com", membership_id="U001")
  lib.users.append(user)

  add_book(
    lib,
    Book(
      title="Clean Code",
      author="Robert C. Martin",
      year=2008,
      categories=["Programming", "Software Engineering", "programming"],
    ),
  )
  add_book(
    lib,
    Book(
      title="Designing Data-Intensive Applications",
      author="Martin Kleppmann",
      year=2017,
      categories=["Data", "Systems"],
    ),
  )

  print("Всего книг:", lib.total_books())
  print("Найти Clean Code:", find_book(lib, "Clean Code"))

  borrowed = is_book_borrow(lib, "Clean Code")
  print("Выдали:", borrowed)

  returned = return_book(lib, "Clean Code")
  print("Вернули:", returned)


2025-10-29 11:35:26,341 [INFO] Operation borrow: started. args=(Library(books=[Book(title='Clean Code', author='Robert C. Martin', year=2008, available=True, categories=['Programming', 'Software Engineering']), Book(title='Designing Data-Intensive Applications', author='Martin Kleppmann', year=2017, available=True, categories=['Data', 'Systems'])], users=[User(name='Alice', email='alice@example.com', membership_id='U001')]), 'Clean Code') kwargs={}
2025-10-29 11:35:26,342 [INFO] Operation borrow: finished. result=title='Clean Code' author='Robert C. Martin' year=2008 available=False categories=['Programming', 'Software Engineering']
2025-10-29 11:35:26,343 [INFO] Operation return: started. args=(Library(books=[Book(title='Clean Code', author='Robert C. Martin', year=2008, available=False, categories=['Programming', 'Software Engineering']), Book(title='Designing Data-Intensive Applications', author='Martin Kleppmann', year=2017, available=True, categories=['Data', 'Systems'])], users=[

Всего книг: 2
Найти Clean Code: title='Clean Code' author='Robert C. Martin' year=2008 available=True categories=['Programming', 'Software Engineering']
Выдали: title='Clean Code' author='Robert C. Martin' year=2008 available=False categories=['Programming', 'Software Engineering']
Вернули: title='Clean Code' author='Robert C. Martin' year=2008 available=True categories=['Programming', 'Software Engineering']
